# Partie principale   

Datasets à utiliser : Yago

**Attention : Dataset dans `/tmp/BDLE_3774877/dataset`**

````
mkdir -p /tmp/BDLE_3774877/dataset
cd /tmp/BDLE_3774877/dataset
tar zxvf /Infos/bd/spark/dataset/freebase/freebase_snippet_surls.tgz
tar zxvf /Infos/bd/spark/dataset/yago/yagoFacts5M.tgz
cp  /Infos/bd/spark/dataset/yago/yagoMiniSample.txt /tmp/BDLE_3774877/dataset
````

In [ ]:
// Extraction des données

case class Triple(sujet: String, prop: String, objet: String)

val dataset = "/tmp/BDLE_3774877/dataset/"
val yagoFile = dataset + "yagoFacts5M.txt"

defined class Triple
dataset: String = /tmp/BDLE_3774877/dataset/
yagoFile: String = /tmp/BDLE_3774877/dataset/yagoFacts5M.txt


In [ ]:
val yago = sc.textFile(yagoFile).
  map(ligne => ligne.split("\\t")).coalesce(8).
  map(tab => Triple(tab(0), tab(1), tab(2))).toDF()

yago.persist
yago.count
yago.show(5)

+------------+----------------+--------------------+
|       sujet|            prop|               objet|
+------------+----------------+--------------------+
|    <!PAUS3>|     <hasGender>|              <male>|
|    <!PAUS3>|<hasMusicalRole>|<wordnet_bass_104...|
|    <!PAUS3>|   <isCitizenOf>|           <Ukraine>|
|    <!PAUS3>|     <wasBornIn>|      <Philadelphia>|
|<!T.O.O.H.!>|     <hasGender>|              <male>|
+------------+----------------+--------------------+
only showing top 5 rows

yago: org.apache.spark.sql.DataFrame = [sujet: string, prop: string ... 1 more field]


## Statistiques de base

### 1) Retourner la liste des 10 propriétés les plus fréquentes. La sortie doit être une liste de couples (prop, freq) triée de manière décroissante.


In [ ]:
val props = yago.groupBy("prop").count.sort($"count".desc).limit(10)

props.show()

+----------------+-------+
|            prop|  count|
+----------------+-------+
|<isAffiliatedTo>|1116512|
|      <playsFor>| 772092|
|   <isCitizenOf>| 731207|
|   <isLocatedIn>| 512925|
|     <hasGender>| 486528|
|     <wasBornIn>| 405252|
|       <actedIn>| 242436|
|        <diedIn>| 131001|
|   <hasWonPrize>| 115476|
| <graduatedFrom>| 112670|
+----------------+-------+

props: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [prop: string, count: bigint]


### 2) Retourner la liste des 10 noeuds ayant le plus grand degré sortant. 
Rappel Le degré sortant d'un noeud n est le nombre de triplets où n est le sujet. 

La sortie doit être une liste de couples (sujet, degré) triée de manière décroissante.

In [ ]:
val noeudsDegresSortant = yago.groupBy("sujet").count.sort($"count".desc).limit(10)
noeudsDegresSortant.show()

+--------------------+-----+
|               sujet|count|
+--------------------+-----+
|       <Ilaiyaraaja>|  878|
|        <Prem_Nazir>|  471|
|     <United_States>|  456|
|   <Deva_(composer)>|  440|
|       <Adoor_Bhasi>|  383|
| <M._S._Viswanathan>|  378|
|         <Mammootty>|  360|
|    <United_Kingdom>|  345|
|<Laxmikant–Pyarelal>|  324|
| <Jagathy_Sreekumar>|  314|
+--------------------+-----+

noeudsDegresSortant: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [sujet: string, count: bigint]


### 3) Pour chaque propriété, retourner le nombre de sujets distincts d'où elle démarre ainsi que le nombre d'objets distincts où elle arrive. 

La sortie doit être une liste de tuples (pro, nb-sujets, nb-objets). Attention Un objet (sujet) peut avoir plusieurs fois la même propriété.

In [ ]:
val propSujets = yago.select("prop", "sujet").distinct.groupBy("prop").count.withColumnRenamed("count", "nbSujet")
//propSujets.show(5)

val propObjets = yago.select("prop", "objet").distinct.groupBy("prop").count.withColumnRenamed("count", "nbObjet")
//propObjets.show(5)

val propStats = propSujets.join(propObjets, "prop")
propStats.show(10)

+--------------------+-------+-------+
|                prop|nbSujet|nbObjet|
+--------------------+-------+-------+
|<hasOfficialLangu...|   1177|    134|
|    <isInterestedIn>|    781|    160|
|         <dealsWith>|    262|    204|
|       <isLocatedIn>| 114649|  30544|
|<hasAcademicAdvisor>|   2318|    527|
|        <isKnownFor>|     68|     56|
|       <isCitizenOf>| 488398|    451|
|        <influences>|   4517|   1967|
|           <exports>|    167|     58|
|           <actedIn>|  35223|  58160|
+--------------------+-------+-------+
only showing top 10 rows

propSujets: org.apache.spark.sql.DataFrame = [prop: string, nbSujet: bigint]
propObjets: org.apache.spark.sql.DataFrame = [prop: string, nbObjet: bigint]
propStats: org.apache.spark.sql.DataFrame = [prop: string, nbSujet: bigint ... 1 more field]


### 4) Encoder la fonction noeudDegre(d:entier) qui retourne les noeuds de degrée d. 

Le degré d'un noeud = degré sortant + degré entrant.

In [ ]:

def noeudDegre(d: Int) : org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]  = {
  // Calcul des degrés entrants
  val noeudsDegresEntrant = yago.groupBy("objet").count.
      where("count < "+d.toString).
      withColumnRenamed("objet", "noeud").
      withColumnRenamed("count", "degEntrant")
  // Calcul des degrés sortants
  val noeudsDegresSortant = yago.groupBy("sujet").count.
      where("count < "+d.toString).
      withColumnRenamed("sujet", "noeud").
      withColumnRenamed("count", "degSortant")
  // Combinaison 
  val noeudsDegres = noeudsDegresEntrant.join(noeudsDegresSortant, "noeud")
  noeudsDegres.where("degSortant + degEntrant = "+d.toString)
  
}


noeudDegre(400).show(10)


+-------------------+----------+----------+
|              noeud|degEntrant|degSortant|
+-------------------+----------+----------+
|<North_Carolina_FC>|       395|         5|
+-------------------+----------+----------+

noeudDegre: (d: Int)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


`$ grep "<North_Carolina_FC>" yagoFacts5M.txt -c
400
`

## Statistiques sur les chemins et co-occurences

### 1) Pour chaque pattern de 2 propriétés qui se suivent, calculer sa fréquence dans les données. 

Exemple : Si le triple pattern (?x,influences,?y) (?y, livesIn, ?z) retourne 1000 résultats alor la fréquence du pattern (influences, livesIn) vaut 1000.

In [ ]:
val objets = yago.select("objet","prop").
                  withColumnRenamed("objet", "y").
                  withColumnRenamed("prop","prop2")
val sujets = yago.select("sujet", "prop").
                  withColumnRenamed("sujet","y").
                  withColumnRenamed("prop","prop1")

val patterns = sujets.join(objets, "y").groupBy("prop1", "prop2").count()

patterns.where("count = 5").show(10)

+--------------------+--------------------+-----+
|               prop1|               prop2|count|
+--------------------+--------------------+-----+
|    <hasMusicalRole>|           <created>|    5|
|     <wroteMusicFor>|           <created>|    5|
|            <diedIn>|              <owns>|    5|
|    <isPoliticianOf>|              <owns>|    5|
|<hasAcademicAdvisor>|          <hasChild>|    5|
|    <isAffiliatedTo>|<hasAcademicAdvisor>|    5|
|        <hasCapital>|       <hasCurrency>|    5|
+--------------------+--------------------+-----+

objets: org.apache.spark.sql.DataFrame = [y: string, prop2: string]
sujets: org.apache.spark.sql.DataFrame = [y: string, prop1: string]
patterns: org.apache.spark.sql.DataFrame = [prop1: string, prop2: string ... 1 more field]


### 2) Encoder la fonction cheminNoeudLongueur(noeud: string, len:entier) qui retourne, pour le sujet noeud, tous les chemins démarrant de noeud et ayant la longueur len. 

La longueur d'un chemin est le nombre de propriétés traversées.

In [ ]:

def cheminNoeudLongueur(noeud: String, len: Int) : Array[org.apache.spark.sql.Row]  = {
  /* Si la longueur voulue est 1, on peut renvoyer directement la requête SQL */
    if (len <= 1) yago.where("sujet like \""+noeud+"\"").collect()
  /* Sinon si la longueur [len] est plus grande, on cherche tous les chemins de longueur [len] - 1, 
    auxquels on ajoute tous les chemins de longueur 1 qui partent de ces chemins.*/
    else cheminNoeudLongueur(noeud, len-1).
          flatMap(row => yago.where("sujet like \""+row.getString(row.length-1)+"\"").collect().
          map(end => Row.fromSeq(row.toSeq.slice(0,row.length-1) ++ end.toSeq)))
}
println("Chemins de longueur 1")
cheminNoeudLongueur("<1-UP_Studio>", 1).take(25).foreach(println)
println("Chemins de longueur 2")
cheminNoeudLongueur("<1-UP_Studio>", 2).take(25).foreach(println)
println("Chemins de longueur 3")
cheminNoeudLongueur("<1-UP_Studio>", 3).take(25).foreach(println)


Chemins de longueur 1
[<1-UP_Studio>,<created>,<Blue_Dragon_Plus>]
[<1-UP_Studio>,<created>,<Mother_(series)>]
[<1-UP_Studio>,<created>,<Super_Mario_3D_World>]
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>]
[<1-UP_Studio>,<isLocatedIn>,<de/Tokio>]
[<1-UP_Studio>,<isLocatedIn>,<Chiyoda,_Tokyo>]
Chemins de longueur 2
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>,<isLocatedIn>,<Japan>]
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>,<isLocatedIn>,<Kantō_region>]
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>,<isLocatedIn>,<Honshu>]
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>,<isLocatedIn>,<Asia>]
[<1-UP_Studio>,<isLocatedIn>,<Chiyoda,_Tokyo>,<isLocatedIn>,<Japan>]
[<1-UP_Studio>,<isLocatedIn>,<Chiyoda,_Tokyo>,<isLocatedIn>,<Kantō_region>]
[<1-UP_Studio>,<isLocatedIn>,<Chiyoda,_Tokyo>,<isLocatedIn>,<Tokyo>]
[<1-UP_Studio>,<isLocatedIn>,<Chiyoda,_Tokyo>,<isLocatedIn>,<Honshu>]
Chemins de longueur 3
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>,<isLocatedIn>,<Japan>,<dealsWith>,<Hong_Kong>]
[<1-UP_Studio>,<isLocatedIn>,<Tokyo>,<isLocatedIn>,<Jap

### 3) Pour chaque paire de propriétés, donner le nombre de sujets qu'elles partagent. 

Exemple. Si le triple pattern (x, livesIn, y) (x, citizenOf, z) retourne 10 résultat alors les propriétés de la paire (livesIn, citizenOf) partagent 10 sujets.

In [ ]:
val prop1 = yago.withColumnRenamed("sujet","x").withColumnRenamed("prop","prop1")
val prop2 = yago.withColumnRenamed("sujet","x").withColumnRenamed("prop","prop2")

val paireFreq = prop1.join(prop2,"x").
      where("prop1 < prop2").
      groupBy("prop1","prop2").
      count().
      withColumnRenamed("count", "sujetsPartages")

paireFreq.orderBy($"sujetsPartages".desc).show(10)

+----------------+----------------+--------------+
|           prop1|           prop2|sujetsPartages|
+----------------+----------------+--------------+
|<isAffiliatedTo>|      <playsFor>|       6017893|
|<isAffiliatedTo>|   <isCitizenOf>|       1655218|
|   <isCitizenOf>|      <playsFor>|       1174284|
|     <hasGender>|<isAffiliatedTo>|       1049416|
|<isAffiliatedTo>|     <wasBornIn>|       1003512|
|     <hasGender>|      <playsFor>|        718536|
|     <hasGender>|   <isCitizenOf>|        714173|
|      <playsFor>|     <wasBornIn>|        685441|
|   <isCitizenOf>|     <wasBornIn>|        594215|
|       <created>| <wroteMusicFor>|        521147|
+----------------+----------------+--------------+
only showing top 10 rows

prop1: org.apache.spark.sql.DataFrame = [x: string, prop1: string ... 1 more field]
prop2: org.apache.spark.sql.DataFrame = [x: string, prop2: string ... 1 more field]
paireFreq: org.apache.spark.sql.DataFrame = [prop1: string, prop2: string ... 1 more field]


## Bonus

** Copie du dataset : **

````tar xzvf /Infos/bd/spark/dataset/dbpedia/dbpediaShortName8M.tgz
cp /Infos/bd/spark/dataset/dbpedia/dbpediaShortNameTypeFor8M.txt /tmp/BDLE_3774877/dataset ````

### 1) Dans un premier temps, compléter les triplets de dbpediaShortName8M avec le type de leurs noeuds qui se trouvent dans dbpediaShortNameTypeFor8M.txt.

In [ ]:
val shortName = dataset + "dbpediaShortName8M.txt"
val shortNameType = dataset + "dbpediaShortNameTypeFor8M.txt"

val dbPedia = sc.textFile(shortName).
  map(ligne => ligne.split("\\t")).coalesce(8).
  map(tab => Triple(tab(0), tab(1), tab(2))).toDS()

println("dbPedia : "+dbPedia.count +" éléments")
dbPedia.persist
dbPedia.show(5)

val dbPediaType = sc.textFile(shortNameType).
  map(ligne => ligne.split("\\t")).coalesce(8).
  map(tab => Triple(tab(0), tab(1), tab(2))).toDS()

println("dbPediaType : "+dbPediaType.count +" éléments")
dbPediaType.persist
dbPediaType.show(5)

val dbPediaAll = dbPedia.union(dbPediaType)

println("dbPediaAll : "+dbPediaAll.count +" éléments")
dbPediaAll.persist
dbPediaAll.show(5)


dbPedia : 8288566 éléments
+-----+----------------+--------------------+
|sujet|            prop|               objet|
+-----+----------------+--------------------+
|<!!!>|<associatedActs>|<Turing_Machine_(...|
|<!!!>|<associatedActs>|           <Out_Hud>|
|<!!!>|<associatedActs>|   <LCD_Soundsystem>|
|<!!!>|<associatedActs>|   <Maserati_(band)>|
|<!!!>|<associatedActs>|  <The_Juan_MacLean>|
+-----+----------------+--------------------+
only showing top 5 rows

dbPediaType : 108145 éléments
+--------------------+------+--------------------+
|               sujet|  prop|               objet|
+--------------------+------+--------------------+
|     <!Hero_(album)>|<type>|      <Double_album>|
|          <$ell.Out>|<type>|             <Album>|
|<%22Ibrahim_Kodra...|<type>|<Non-governmental...|
|<%22No_Snow,_No_S...|<type>|        <Live_album>|
|<%22Ridgeriders%2...|<type>|        <Live_album>|
+--------------------+------+--------------------+
only showing top 5 rows

dbPediaAll : 8396711

### 2) Pour chaque type, retourner son domaine, i.e le nombre de sujets distincts ayant ce type.

In [ ]:
val domaines = dbPediaAll.select("sujet","objet").
    where("prop = '<type>'").
    groupBy("objet").
    count.
    withColumnRenamed("objet", "type").
    withColumnRenamed("count", "domaine")

domaines.orderBy($"domaine".desc).show(10)

+--------------------+-------+
|                type|domaine|
+--------------------+-------+
|             <Album>|  26044|
|      <State_school>|   6207|
| <Public_university>|   3907|
|    <Private_school>|   3674|
|<Privately_held_c...|   3127|
|<Private_university>|   2314|
|        <Live_album>|   2276|
|<Mixed-sex_educat...|   1991|
|    <Public_company>|   1978|
| <Compilation_album>|   1841|
+--------------------+-------+
only showing top 10 rows

domaines: org.apache.spark.sql.DataFrame = [type: string, domaine: bigint]


### 3) Pour chaque type, retourner son co-domaine, i.e le nombre d'objets distincts ayant ce type.

Il faut trouver tous les objets distincts dans la table ainsi que leurs types.

In [ ]:
// Tous les sujets et leur type
val types = dbPediaAll.select("sujet","objet").
      where("prop = '<type>'").
      withColumnRenamed("objet","type").
      withColumnRenamed("sujet","objet")

// Tous les objets qui ne sont pas des types
val objets = dbPediaAll.select("objet").where("prop != '<type>'")


val res = types.join(objets,"objet").distinct.groupBy("type").count

res.orderBy($"count".desc).show(10)

+--------------------+-----+
|                type|count|
+--------------------+-----+
| <Public_university>| 1611|
|<Private_university>|  745|
|    <Private_school>|  727|
|             <Album>|  677|
|      <State_school>|  406|
|    <Public_company>|  347|
|<Privately_held_c...|  345|
|<Promotional_reco...|  264|
|<Mixed-sex_educat...|  200|
|    <Beauty_pageant>|  159|
+--------------------+-----+
only showing top 10 rows

types: org.apache.spark.sql.DataFrame = [objet: string, type: string]
objets: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [objet: string]
res: org.apache.spark.sql.DataFrame = [type: string, count: bigint]


On peut vérifier pour le type `<Private_school>` par exemple :

On récupère tous les sujets du type voulu, on stocke dans un fichier :

> `grep ".*\s*<type>\s*<Private_school>" dbpediaShortNameTypeFor8M.txt | cut  -f1 > priv.txt`

> `grep -c ".*" priv.txt `

> 3674


On cherche ces sujets en tant qu'objet dans le fichier dbPedia original, on stock le résultat dans un fichier :

> `while read line; do grep "<.*>\s*<.*>\s*$line" dbpediaShortName8M.txt ; done < priv.txt > priv2.txt`

On récupère le nombre d'objets distincts :

> `cut priv2.txt -f3 | uniq -c | grep -c ".*"` 

> 727